# $\langle XXXXXXXX\rangle$ Stabilizer example
    
Adapted from Ignis example by Chris Wood

In [1]:
import numpy as np
from qiskit import *
from qiskit.providers.aer import QasmSimulator, noise

from qiskit.ignis.mitigation import (expectation_value, 
                                     expval_meas_mitigator_circuits,
                                     ExpvalMeasMitigatorFitter)

import mthree

## Load systems and simulators

In [2]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
pro= IBMQ.get_provider(project='internal-test')

In [4]:
sim = QasmSimulator()
backend = pro.backends.ibmq_dublin

<ipython-input-4-cae98a4f3b2a>:2: DeprecationWarning: The `backends` provider attribute is deprecated. Please use `provider.backend` (singular) instead. You can continue to use `provider.backends()` to retrieve all backends.
  backend = pro.backends.ibmq_dublin


In [5]:
# Test Circuit Measure <XXXXXXXX> stabilizer
# Eight qubit stabilizer
qubits=[1, 4, 7, 10, 12, 2, 3, 5]

qc = QuantumCircuit(8, 8)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(2, 3)
qc.cx(3, 4)
qc.cx(0, 5)
qc.cx(5, 6)
qc.cx(6, 7)
qc.h(range(8))
qc.measure(range(8), range(8))

shots = 8192
#Get ideal result and raw counts from backend
result_target = execute(qc, sim, shots=shots, initial_layout=qubits).result()
result_noise = execute(qc, backend, shots=shots, initial_layout=qubits).result()
counts_target = result_target.get_counts(0)
counts_noise = result_noise.get_counts(0)

## Apply M3 correction

In [6]:
mit = mthree.M3Mitigation(backend)

In [7]:
mit.tensored_cals_from_system(qubits)

In [18]:
mit_counts, details = mit.apply_correction(counts_noise, qubits=qubits, details=True)
details

{'method': 'direct',
 'time': 0.004311084747314453,
 'dimension': 256,
 'col_norms': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
    

## Apply CTMP correction

In [9]:
circuits1, metadata1 = expval_meas_mitigator_circuits(len(qubits), method='tensored')

In [10]:
job_cal1 = execute(circuits1, backend, shots=8192, initial_layout=qubits)
result_cal1 = job_cal1.result()

In [11]:
ctmp_mitigator = ExpvalMeasMitigatorFitter(result_cal1, metadata1).fit()

## Tally all results

In [19]:
print('Ideal:', expectation_value(counts_target)[0])
print('Raw:', expectation_value(counts_noise)[0])
print('CTMP mitigated:', expectation_value(counts_noise, meas_mitigator=ctmp_mitigator)[0])
print('M3 mitigated:', expectation_value(mit_counts)[0])

Ideal: 1.0
Raw: 0.497802734375
CTMP mitigated: 1.3377458081929379
M3 mitigated: 0.9543008693805328
